Neste exemplo você irá construir um classificador binário para o dataset do IMDB (NLP Problem)

O imdb dataset já vem com o Keras e já é pré-processado. Os reviews (sequência de palavras) já estão organizados em sequências de inteiros, em que cada inteiro significa uma palavra específica no dicionário.


In [ ]:
from keras.datasets import imdb

As 10000 palavras mais frequentes serão mantidas no dataset

In [ ]:
# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Train_data e test_data são as listas de reviews, cada review é uma lista de índices de palavras (texto do review codificado no índice de palavras)


In [ ]:
print(train_data[6])

Train_labels e test_labels são as listas de 0’s e 1’s (0 = negativo review e 1 = positivo review)


In [ ]:
print(train_labels[6])

Veja que nenhum índice de palavra irá exceder 10000

In [ ]:
max([max(sequence) for sequence in train_data])

Não podemos inserir um lista de inteiros na rede. Temos que converter a listar para um tensor no formato (samples, word_indices)

Por exemplo, transformar a sequência [3, 5] em um vetor de 10.000 dimensões que seria todos os 0s, exceto os índices 3 e 5, que seriam 1s.

In [ ]:
import numpy as np

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
  # Creates an all-zero matrix of shape (len(sequences), dimension)
  results = np.zeros((len(sequences), dimension))

  for i, sequence in enumerate(sequences):
    # Sets specific indices of results[i] to 1s
    results[i, sequence] = 1.

  return results

In [ ]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
print(x_train[0])
print(x_train.shape)

O mesmo para os vetores de labels


In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

**Construindo a rede**

O input é um vetor e os labels são escalares (1’s e 0’s). Vamos usar um fully connected (Dense) com relu como activation function.

Dense(16, activation='relu’)

16: é o argumento para cada dense layer. Ou seja, é o número de hidden units da camada (é a dimensão da representação da camada). 


**Configurando a rede**

Iremos configurar a rede com duas camadas intermediárias com 16 unidades ocultas cada.
Uma terceira camada que produzirá a previsão escalar em relação ao sentimento do review em questão

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Escolhendo a loss function e o optimizer (como strings por já fazerem parte do Keras)


In [ ]:
from tensorflow.keras import losses
from tensorflow.keras import metrics
# from keras.optimizers import SGD

# model.compile(optimizer=RMSprop(lr=0.001),
# loss=losses.binary_crossentropy,
# metrics=[metrics.binary_accuracy])

No entanto, você pode deixar tudo como padrão:

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

Para monitorar o treinamento, criamos o conjunto de validação separando 10000 amostras do conjunto original

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Iremos treinar o modelo por 20 épocas (20 iterações sobre todas as amostras do treinamento) em um batch de 512 amostras.


In [ ]:
model.fit(partial_x_train,
partial_y_train,
epochs=100,
batch_size=512,
validation_data=(x_val, y_val))

**Usando uma rede treinada para gerar previsões sobre novos dados**

Gerando a probabilidade de cada análise ser positiva com o método predict

In [ ]:
print(model.predict(x_test))

A rede está confiante para algumas amostras (0,99 ou mais, ou 0,01 ou menos), mas menos confiante para outras (0,6, 0,4).

**Treinando o modelo do zero**

Observe que no treino abaixo não separamos o conjunto de treino em treino e validação, uma vez que não iremos validar o modelo durante o treinamento e sim após o mesmo com o model.evaluate()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results = model.evaluate(x_test, y_test)

Referência: François Chollet. Deep Learning with Python. November 2017  